# The LineCollection Object 

Emission lines are bright features in a spectrum caused by specific atomic transitions. In astronomy, these lines are often used to identify the presence of certain elements in celestial objects, such as galaxies or nebulae. The luminosity of these lines can vary significantly, and they can be influenced by various factors such as redshift, dust extinction, and the physical conditions of the emitting gas. 

In Synthesizer emission lines are encapsulated by the ``LineCollection`` object. This object holds an arbitrary collection of emission lines, with each luminosity, continuum, line ID, and wavelength stored in arrays. However, as well as providing an interface to these arrays, the ``LineCollection`` can also be treated as a dictionary. 

In this section, we will demonstrate how to create a ``LineCollection`` object, how to access its properties, and how to manipulate it.

## Creating a LineCollection

Before demonstrating anything, we need a ``LineCollection`` to demonstrate with. A ``LineCollection`` can be created from scratch, taking user specified luminosities, continuum, line IDs, and wavelengths.

In [ ]:
from unyt import Hz, Myr, angstrom, erg, s

In [ ]:
from synthesizer import LineCollection

lines = LineCollection(
    "ArbitraryLine",
    lam=912 * angstrom,
    lum=1e28 * erg / s,
    cont=1e28 * erg / s / Hz,
)

However, in practice, it will be far more common to use ``LineCollection`` objects produced by or derived from other Synthesizer objects (e.g. from combining a ``Galaxy`` and an ``EmissionModel``). 

For the rest of this section, we will take lines directly from a ``Grid`` object (more information [here](../../emission_grids/grids.rst)). For more details on line generation see the [grid lines](grid_lines.ipynb) and [galaxy lines](galaxy_lines.ipynb) notebooks for thorough demonstrations of how to generate lines from each.

In [ ]:
from synthesizer.grid import Grid

grid = Grid("test_grid")

# Extract the lines for a specific point in the grid
lines = grid.get_lines(
    grid.get_grid_point(log10ages=7.0 * Myr, metallicity=0.01)
)

## Printing a summary of the LineCollection 

If we want a summary of what a line contains we can simply print it. This will give us a table showing all the different attributes attached to a ``LineCollection`` object and their values (or a summary of their values for large arrays).

In [ ]:
print(lines)
print(lines.line_ids)

## Accessing specific lines

A ``LineCollection`` behaves somewhat like a dictionary, we can access line data by passing strings when we index the ``LineCollection``. Unlike a dictionary, however, we are not limited to singular keys, we can pass multiple at once in a ``list`` or ``tuple`` to instead return a subset of lines. Not only that, we can also defined composite lines by passing a string of comma separated lines. We demonstrate each of these methods to axis data below.

Line IDs follow the same convention as in Cloudy, whereby a line is usually represented as ``{atomic or molecular notation} {ionisation state} {wavelength}``. The ionisation state is in the usual astronomical notation, e.g. H 1 for atomic hydrogen, but different for molecules, with the state denoted by '+' or '-' (e.g. HCO+).

> Note that in reality the data is actually stored in contiguous arrays for efficiency.

### Extracting a single line

To extract a single line we simply pass the name of the line we want to extract.

In [ ]:
print(lines["H 1 1215.67A"])

### Extracting composite lines

To extract a composite line (e.g. a doublet or triplet) we pass a string of comma separated lines. This will return a new ``LineCollection`` object with the composite line as the only line in the collection.

In [ ]:
print(lines["Mg 7 2628.89A, Fe 2 2631.05A"])  # doublet
print(lines["Mg 7 2628.89A, Fe 2 2631.05A, Fe 2 2631.32A"])  # triplet

### Extracting a subset of lines

To extract a subset of lines we pass a list or tuple of the names of the lines we want to extract.

In [ ]:
print(
    lines[("Mg 7 2628.89A", "Fe 2 2631.05A", "Fe 2 2631.32A", "Mg 5 2782.76A")]
)

Note that this list can include composite lines as well as single lines.

In [ ]:
print(
    lines[("Mg 7 2628.89A, Fe 2 2631.05A", "Fe 2 2631.32A", "Mg 5 2782.76A")]
)

### Line aliases 

Any of these operations can also be performed by passing some common line aliases defined in the emissions submodule.

In [ ]:
from synthesizer.emissions import line_aliases

for alias in line_aliases:
    print(f"{alias} -> {line_aliases[alias]}")

# We can use any of these aliases in place of the long form ID
print(lines["Hb"])

## Getting line ratios

In addition to the line extraction methods above, we can also get known ratios by passing the ratio name as an index. These ratios are defined in the ``line_ratios`` submodule (details in the [line ratios](line_ratios.ipynb) notebook).

In [ ]:
ratio = lines["BalmerDecrement"]
print(ratio)

Alternatively, we could use the ``get_ratio`` method. This can also take a ratio name but also enables the passing of a list of two lines to get the ratio of.

In [ ]:
ratio2 = lines.get_ratio("BalmerDecrement")
ratio3 = lines.get_ratio(["H 1 4861.32A", "H 1 6562.80A"])
print(ratio2)
print(ratio3)

## Getting diagnostic diagrams

We can also get diagnostic diagrams by passing the name of the diagram as an index. These diagrams are are also defined in the ``line_ratios`` submodule (details in the [line ratios](line_ratios.ipynb) notebook).

In [ ]:
dia = lines["BPT-NII"]
print(dia)

Or we can use the ``get_diagram`` method to get a diagnostic diagram by passing the name of the diagram or a list of lists of lines.

In [ ]:
dia2 = lines.get_diagram("BPT-NII")
print(dia2)
dia3 = lines.get_diagram(
    [["O 3 5006.84A", "H 1 4861.32A"], ["N 2 6583.45A", "H 1 6562.80A"]]
)
print(dia3)

## Arithmetic operations

We can perform various arithmetic operations on a ``LineCollection`` object. These operations are performed element-wise on the underlying line data.


In [ ]:
# Get a subset for the demonstration
lines_subset = lines[("Fe 2 2631.05A", "Fe 2 2631.32A")]


### Adding LineCollections 

We can add two ``LineCollection`` objects together. This will return a new ``LineCollection`` object with the same lines as the two input ``LineCollection`` objects, but with the line data added together.

In [ ]:
new_line = lines_subset + lines_subset
print(new_line.luminosity / lines_subset.luminosity)

### Scaling a LineCollection

We can scale (multiply) a ``LineCollection`` object by a scalar. This will return a new ``LineCollection`` object with the same lines as the input ``LineCollection`` object, but with the line luminosity scaled accordingly.

In [ ]:
new_lines = lines_subset * 4
print(new_lines.luminosity / lines_subset.luminosity)

## Iterating over a LineCollection

We can loop over the individual lines just as we would loop over an array or list. Here we will demonstrate this on a subset.

In [ ]:
lines_subset = lines[
    ("Mg 7 2628.89A", "Fe 2 2631.05A", "Fe 2 2631.32A", "Mg 5 2782.76A")
]
for line in lines_subset:
    print(
        f"ID: {line.id}, Wavelength: {line.lam}, "
        f"Luminosity: {line.luminosity}, Continuum: {line.continuum}"
    )

## Computing fluxes

By default a line contains the rest frame luminosity and continuum luminosity. If we instead want the flux we can use the ``get_flux`` or ``get_flux0`` methods which each populate the ``flux`` and ``continuum_flux`` attributes on the line. The latter of these will compute the rest frame flux at a distance of 10pc.

In [ ]:
lines.get_flux0()
print(lines["H 1 4861.32A"].flux, lines["H 1 4861.32A"].continuum_flux)

While the former requires an Astropy cosmology object and a redshift to compute the observer frame flux.

In [ ]:
from astropy.cosmology import Planck18 as cosmo

lines.get_flux(cosmo, 8.0)
print(lines["H 1 4861.32A"].flux, lines["H 1 4861.32A"].continuum_flux)

## Concatenating LineCollections

If rather than adding the values of two ``LineCollection`` objects together we want to instead concatenate them along the first axis, we can use the ``concatenate`` method. This will return a new ``LineCollection`` object with all the lines from the two input ``LineCollection`` objects.

In [ ]:
import numpy as np
from unyt import Hz, angstrom, erg, s

from synthesizer.emissions import LineCollection

# Simulating having two sets of ndim=2 lines we need to combine
lines1 = LineCollection(
    line_ids=["O 3 5006.84A", "H 1 4861.32A"],
    lam=[5006.84 * angstrom, 4861.32 * angstrom],
    lum=np.array([[1.0, 2.0], [3.0, 4.0]]) * erg / s,
    cont=np.array([[0.1, 0.2], [0.3, 0.4]]) * erg / s / Hz,
)
lines2 = LineCollection(
    line_ids=["O 3 5006.84A", "H 1 4861.32A"],
    lam=[5006.84 * angstrom, 4861.32 * angstrom],
    lum=np.array([[5.0, 6.0], [7.0, 8.0]]) * erg / s,
    cont=np.array([[0.5, 0.6], [0.7, 0.8]]) * erg / s / Hz,
)
print(lines1.shape, lines2.shape)

lines3 = lines1.concat(lines2)
print(lines3.shape)

## Extending a LineCollection with new lines

If we want to add new lines to a ``LineCollection`` object we can use the ``extend`` method. This will return a new ``LineCollection`` object with all the lines from the original ``LineCollection`` object and the new lines. Note that the shape of the new lines must match the shape of the existing lines.

In [ ]:
# Set up two line collections with different lines
lines1 = LineCollection(
    line_ids=["O 3 5006.84A", "H 1 4861.32A"],
    lam=[5006.84 * angstrom, 4861.32 * angstrom],
    lum=np.array([[1.0, 2.0], [3.0, 4.0]]) * erg / s,
    cont=np.array([[0.1, 0.2], [0.3, 0.4]]) * erg / s / Hz,
)
lines2 = LineCollection(
    line_ids=["N 2 6583.45A", "H 1 6562.80A"],
    lam=[6583.45 * angstrom, 6562.80 * angstrom],
    lum=np.array([[5.0, 6.0], [7.0, 8.0]]) * erg / s,
    cont=np.array([[0.5, 0.6], [0.7, 0.8]]) * erg / s / Hz,
)
print(lines1.shape, lines2.shape)

lines3 = lines1.extend(lines2)
print(lines3.shape)

## Blending lines

Lines in a ``LineCollection`` can be blended based on a given wavelength resolution using the ``get_blended_lines`` method. This method takes a set of wavelength bins, and returns a new ``LineCollection`` containing lines blended within each bin.

In [ ]:
import numpy as np
from unyt import angstrom

# Get a subset of lines to blend
lines_subset = lines["H 1 4861.32A", "O 3 5006.84A", "O 3 4958.91A"]

print("Before blending:")
print(lines_subset)

# Blend the lines  onto an arbitrary wavelength grid
lam_bins = np.arange(4000, 7000, 1000) * angstrom
blended_lines = lines_subset.get_blended_lines(lam_bins)

print("After blending:")
print(blended_lines)

## Plotting Lines

To plot lines you can use a ``LineCollection`` instance's ``plot_lines`` method. This can be incomprehensible if you plot all lines so we'll pass a subset to the subset argument.

In [ ]:
_ = lines.plot_lines(
    figsize=(12, 5),
    subset=[
        "He 2 1025.27A",
        "He 2 1084.94A",
        "Si 2 1179.59A",
        "Si 3 1206.50A",
        "He 2 1215.13A",
        "H 1 1215.67A",
        "Si 2 1264.74A",
        "O 1 1302.17A",
        "O 1 1304.86A",
    ],
    xlimits=(None, 1350),
    ylimits=(10**28.0, 10**36.0),
)

## Line Ratios

To help with the analysis of emission lines, Synthesizer provides a submodule called ``line_ratios``. This submodule contains definitions for commonly used line ratios and diagnostic diagrams that can be derived from the emission lines in a ``LineCollection``.

We can print out the available line ratios with the ``available_ratios`` constant.

In [ ]:
from synthesizer.emissions import line_ratios

print(line_ratios.available_ratios)

# Print the ratios and their definitions
for ratio in line_ratios.available_ratios:
    print(
        f"{ratio} = {line_ratios.ratios[ratio][0]} "
        f"/ {line_ratios.ratios[ratio][1]}"
    )

## Diagnostic Diagrams

In addition to line ratios, the submodule also provides a set of common diagnostic diagrams that can be used, for example, to classify the ionizing source.

In [ ]:
print(line_ratios.available_diagrams)

for diagram in line_ratios.available_diagrams:
    print(
        f"{diagram} = {line_ratios.diagrams[diagram][0]} "
        f"/ {line_ratios.diagrams[diagram][1]}"
    )

## Combining ``line_ratios`` with a ``LineCollection``



In [ ]:
from synthesizer.grid import Grid

grid = Grid("test_grid")
lines = grid.get_lines((1, 8))

As shown in the [line collection docs](lines_example.ipynb), we can measure some predefined line ratios by passing the name of a ratio. We can also import all line ratios and loop over all pre-defined ratios.

In [ ]:
for ratio_id in lines.available_ratios:
    ratio = lines.get_ratio(ratio_id)
    print(f"{ratio_id}: {ratio:.2f}")

Similarly, we can also do same for all the diagnostic diagrams.

In [ ]:
for diagram_id in lines.available_diagrams:
    diagram0, diagram1 = lines.get_diagram(diagram_id)
    print(f"{diagram_id}: {diagram0:.2f} / {diagram1:.2f}")

## BPT Classification Regions

The ``lines_ratios`` module also provides functions defining BPT classification regions from [Kewley+13](https://ui.adsabs.harvard.edu/abs/2013ApJ...774L..10K/abstract) and [Kauffmann+03](https://ui.adsabs.harvard.edu/abs/2003MNRAS.346.1055K/abstract). These functions take a set of x values and return the y values that define the classification regions.

In [ ]:
import numpy as np

logNII_Ha = np.arange(-2.0, 1.0, 0.01)
logOIII_Hb_kewley = line_ratios.get_bpt_kewley01(logNII_Ha)
logOIII_Hb_kauffman = line_ratios.get_bpt_kauffman03(logNII_Ha)

We also provide functions to plot these classifications either on an existing set of axes or on a new figure.

In [ ]:
fig, ax = line_ratios.plot_bpt_kewley01(
    logNII_Ha, show=False, fig=None, ax=None, label="Kewley+01"
)
_, _ = line_ratios.plot_bpt_kauffman03(
    logNII_Ha, fig=fig, ax=ax, label="Kauffman+03", show=True
)



For a more interesting example, see the metallicity dependence plots in the [grid lines example](grid_lines.ipynb).